In [ ]:
!pip install feast[aws] pandas xgboost

     |████████████████████████████████| 289 kB 5.3 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 100 kB 10.2 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 2.3 MB 49.5 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 10.9 MB 38.7 MB/s 
     |████████████████████████████████| 132 kB 62.3 MB/s 
     |████████████████████████████████| 146 kB 45.6 MB/s 
     |████████████████████████████████| 8.6 MB 33.8 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 138 kB 79.1 MB/s 
     |████████████████████████████████| 134 kB 74.5 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████

In [ ]:
import pandas as pd
# from datetime import datetime, timedelta

##Read the data and filter out data that belongs to country other than UK
retail_data = pd.read_csv('/content/OnlineRetail.csv', encoding= 'unicode_escape')
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], errors = 'coerce')
uk_data = retail_data.query("Country=='United Kingdom'").reset_index(drop=True)

In [ ]:
entity_df = uk_data[["CustomerID"]]
len(entity_df.CustomerID.unique())

3951

In [ ]:
t1 = pd.Timestamp("2011-06-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
uk_data_3m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
entity_df = uk_data_3m[["CustomerID"]]
len(entity_df.CustomerID.unique())

1841

In [ ]:
from datetime import datetime
entity_df = pd.DataFrame(data = {
    "CustomerID": [str(item) for item in uk_data_3m.CustomerID.unique().tolist()],
    "event_timestamp": datetime.now()
})
entity_df.head()

,CustomerID,event_timestamp
0,14620.0,2022-03-27 01:14:32.527633
1,14740.0,2022-03-27 01:14:32.527633
2,13880.0,2022-03-27 01:14:32.527633
3,16462.0,2022-03-27 01:14:32.527633
4,17068.0,2022-03-27 01:14:32.527633


In [ ]:
%cd customer_segmentation

#import feast and load feature store object with the path to the directory which contains feature_story.yaml.
from feast import FeatureStore
store = FeatureStore(repo_path=".")
for entity in store.list_entities():
  print(f"entity: {entity}")

/content/customer_segmentation
Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage


/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]


entity: {
  "spec": {
    "name": "customer",
    "valueType": "STRING",
    "description": "Id of the customer",
    "joinKey": "CustomerID"
  },
  "meta": {
    "createdTimestamp": "2022-03-14T23:41:33.070471Z",
    "lastUpdatedTimestamp": "2022-03-14T23:41:33.070471Z"
  }
}


In [ ]:
feature_view = store.get_feature_view("customer_rfm_features")
print(feature_view.to_proto())

spec {
  name: "customer_rfm_features"
  entities: "customer"
  features {
    name: "Recency"
    value_type: INT32
  }
  features {
    name: "Frequency"
    value_type: INT32
  }
  features {
    name: "MonetaryValue"
    value_type: DOUBLE
  }
  features {
    name: "R"
    value_type: INT32
  }
  features {
    name: "F"
    value_type: INT32
  }
  features {
    name: "M"
    value_type: INT32
  }
  features {
    name: "RFMScore"
    value_type: INT32
  }
  features {
    name: "Revenue6m"
    value_type: DOUBLE
  }
  features {
    name: "LTVCluster"
    value_type: INT32
  }
  features {
    name: "SegmentHighValue"
    value_type: INT32
  }
  features {
    name: "SegmentLowValue"
    value_type: INT32
  }
  features {
    name: "SegmentMidValue"
    value_type: INT32
  }
  ttl {
    seconds: 315360000
  }
  batch_source {
    type: BATCH_REDSHIFT
    event_timestamp_column: "event_timestamp"
    created_timestamp_column: "created_timestamp"
    redshift_options {
      query

/usr/local/lib/python3.7/dist-packages/feast/infra/offline_stores/redshift_source.py:55: DeprecationWarning: Starting in Feast 0.21, Feast will require either a name for a data source (if using query) or `table`.
  DeprecationWarning,


In [ ]:
import os
from datetime import datetime
os.environ["AWS_ACCESS_KEY_ID"] = "<aws_key>"
os.environ["AWS_SECRET_ACCESS_KEY"] = "<aws_secret>"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
              "customer_rfm_features:recency", 
              "customer_rfm_features:MonetaryValue", 
              "customer_rfm_features:R", 
              "customer_rfm_features:M",
              "customer_rfm_features:RFMScore",
              "customer_rfm_features:SegmentHighValue",
              "customer_rfm_features:SegmentLowValue",
              "customer_rfm_features:SegmentMidValue",
              "customer_rfm_features:LTVCluster"
              ]
    )
feature_data = job.to_df()
feature_data = feature_data.dropna()
feature_data.head()

/usr/local/lib/python3.7/dist-packages/feast/infra/offline_stores/redshift_source.py:55: DeprecationWarning: Starting in Feast 0.21, Feast will require either a name for a data source (if using query) or `table`.
  DeprecationWarning,


,customerid,event_timestamp,recency,monetaryvalue,r,m,rfmscore,segmenthighvalue,segmentlowvalue,segmentmidvalue,ltvcluster
0,12747.0,2022-03-27 01:14:32.527633,7.0,1082.09,3.0,3.0,8.0,1.0,0.0,0.0,0.0
2,12841.0,2022-03-27 01:14:32.527633,31.0,548.34,1.0,2.0,6.0,0.0,0.0,1.0,0.0
3,12849.0,2022-03-27 01:14:32.527633,77.0,704.50,0.0,2.0,4.0,0.0,1.0,0.0,0.0
4,12854.0,2022-03-27 01:14:32.527633,50.0,401.82,1.0,2.0,5.0,0.0,0.0,1.0,0.0
5,12863.0,2022-03-27 01:14:32.527633,7.0,-17.00,3.0,0.0,3.0,0.0,1.0,0.0,0.0


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# feature_data = pd.get_dummies(merged_data)
X = feature_data.drop(['ltvcluster', 'customerid', 'event_timestamp'], axis=1)
y = feature_data['ltvcluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
xgb_classifier = xgb.XGBClassifier(max_depth=5, objective='multi:softprob')
xgb_model = xgb_classifier.fit(X_train, y_train)
acc = xgb_model.score(X_test,y_test)
print(f"Model accuracy: {acc}")

Model accuracy: 0.9202898550724637


In [ ]:
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96       117
         1.0       0.50      0.25      0.33         4
         2.0       0.83      0.59      0.69        17

    accuracy                           0.92       138
   macro avg       0.76      0.61      0.66       138
weighted avg       0.91      0.92      0.91       138



In [ ]:
!pip install joblib

In [ ]:
import joblib
joblib.dump(xgb_model, '/content/customer_segment-v0.0')

['/content/customer_segment-v0.0']

In [ ]:
loaded_model = joblib.load('/content/customer_segment-v0.0')
prediction = loaded_model.predict(X_test.head())
prediction.tolist()

[0.0, 0.0, 0.0, 2.0, 0.0]

In [ ]:
import boto3
s3_client = boto3.client('s3')
response = s3_client.upload_file('/content/customer_segment-v0.0', "feast-demo-mar-2022", "model-repo/customer_segment-v0.0")

None
